Bismillah Bisa

Loading Data

In [1]:
import pandas as pd
import numpy as np

df = pd.read_excel("C:/Users/Mirtha/Downloads/Source Code Skripsi/Data Labelling Sendiri.xlsx")
df=df.fillna('')
df

,Nama Fitur,User Story,Scenario,Scenario.1,Scenario.2,Scenario.3,Scenario.4,Scenario.5,Scenario.6,Scenario.7,Scenario.8,Scenario.9,Scenario.10,Effort
0,Create Program Outcomes,In order to establish Program Outcomes\nAs Cur...,Scenario: Create Program Outcomes Successfully...,Scenario: Failed to create Program Outcomes as...,Scenario: Failed to Create Program Outcomes Du...,,,,,,,,,M
1,Read Program Outcomes,In order to evaluate Program Outcomes\nAs Curr...,Scenario: Program Outcomes Provided\nGiven I a...,Scenario: Empty Program Outcomes\nGiven I am o...,Scenario: Program Outcomes Export to Excel\nGi...,Scenario: Program Outcomes Export to PDF\nGive...,,,,,,,,M
2,Update Program Outcomes,In order to improve Program Outcomes\nAs Curri...,Scenario: Update Program Outcomes Successfully...,Scenario: Failed to update Program Outcomes as...,Scenario: Failed to update Program Outcomes Du...,,,,,,,,,M
3,Delete Program Outcomes,In order to remove irrelevant Program Outcomes...,Scenario: Delete Program Outcomes Successfully...,Scenario: Failed to delete Program Outcomes Du...,,,,,,,,,,S
4,Create SNDikti Learning Outcomes,In order to adapt SNDikti Learning Outcomes\nA...,Scenario: Create SNDikti Learning Outcome Succ...,Scenario: Failed to create SNDikti Learning Ou...,Scenario: Failed to Create SNDikti Learning Ou...,,,,,,,,,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,Read Study Program,In order to verify Study Program Data\nAs Curr...,Scenario: Study Program Provided\nGiven I am o...,,,,,,,,,,,S
69,Update Study Program,In order to improve Study Program Data \nAs Cu...,Scenario: Update Study Program Successfully\nG...,Scenario: Failed to update Study Program Due t...,,,,,,,,,,M
70,Read User Management,"In order to verify User\nAs Curriculum Team,\n...",Scenario: User Provided\nGiven I am on “http:/...,,,,,,,,,,,M
71,Update User Management,In order to improve User Data \nAs Curriculum ...,Scenario: Update User Successfully\nGiven I am...,Scenario: Failed to update User Due to Empty D...,,,,,,,,,,M


Menggabungkan data

In [ ]:
df['combined'] = df.iloc[:, :13].apply(lambda x: '\n'.join(x), axis=1)

# Menampilkan hasil
print(df.loc[0, 'combined'])

Menghitung Jumlah Skenario dan Kata

In [ ]:
df['combined'] = df['combined'].str.lower()
df['number_word'] = df['combined'].apply(lambda x: len(x.split()))
df['number_scenario'] = df['combined'].apply(lambda x: x.count("scenario:"))
df['number_word']
df['number_scenario'] 

Normalisasi Fitur Numerik

In [ ]:
from sklearn.preprocessing import MinMaxScaler
# Inisialisasi MinMaxScaler
scaler = MinMaxScaler()
# Normalisasi 'number_word' dan 'number_scenario' dengan MinMaxScaler
df[['number_word', 'number_scenario']] = scaler.fit_transform(df[['number_word', 'number_scenario']])
df[['number_word', 'number_scenario']]

Remove Punctuation

In [ ]:
import string

# Function to remove punctuation
def remove_punctuation(text):
    punctuation_to_replace = string.punctuation + "‘’“”"
    translation_table = str.maketrans(punctuation_to_replace, ' ' * len(punctuation_to_replace))
    cleaned_text = text.translate(translation_table)
    return cleaned_text

# Assuming df is your DataFrame
df['combined'] = df['combined'].apply(remove_punctuation)
print(df.loc[0, 'combined'])

Tokenizer

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Load English stopwords
stop_words = set(stopwords.words('english'))

# Lemmatizer initialization
lemmatizer = WordNetLemmatizer()

# Tokenize the text in the 'combined' column
df['combined'] = df['combined'].apply(lambda x: word_tokenize(x))
print(df['combined'])

Stopwords Removal

In [ ]:
# Function to remove stopwords
def remove_stopwords(tokens):
    return [token for token in tokens if token.lower() not in stop_words]

# Remove stopwords from the tokenized text
df['combined'] = df['combined'].apply(remove_stopwords)
df['combined']

Lematisasi

In [ ]:
# Define stopwords and lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to perform lemmatization
def lemmatize_tokens(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

df['combined'] = df['combined'].apply(lemmatize_tokens)
df['combined']

TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Convert lemmatized tokens back to strings
df['combined'] = df['combined'].apply(lambda x: ' '.join(x))

# Initialize TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the lemmatized text
tfidf_matrix = tfidf_vectorizer.fit_transform(df['combined'])

# Convert the TF-IDF matrix to a DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
tfidf_df

Data Integration

In [ ]:
# Concatenate 'effort' column of the original DataFrame with TF-IDF DataFrame
integrated_data = pd.concat([tfidf_df, df[['number_word', 'number_scenario', 'Effort']]], axis=1)
integrated_data

One hot encoder

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# One-Hot Encoding for 'Effort'
onehot_encoder = OneHotEncoder()
effort_encoded = onehot_encoder.fit_transform(integrated_data['Effort'].values.reshape(-1,1))
effort_encoded_array = effort_encoded.toarray()

# Mengambil indeks kolom dengan nilai maksimum untuk setiap baris
effort_labels = np.argmax(effort_encoded_array, axis=1)

In [ ]:
# Melihat kategori yang diencode
encoded_categories = onehot_encoder.categories_
print("Encoded Categories:", encoded_categories)

# Mencetak nilai yang sesuai untuk setiap kategori
for i, category in enumerate(encoded_categories[0]):
    print(f"Kategori '{category}' diencode menjadi:", i)


Splitting Data

In [ ]:
# Splitting data into features (X) and target variable (y)
X = integrated_data.drop(columns=['Effort'])
y = effort_encoded_array

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Splitting data into features (X) and target variable (y)
X = integrated_data.drop(columns=['Effort'])
y = effort_labels

# Define AdaBoost Classifier
adaboost_model = AdaBoostClassifier()

# Perform k-fold cross-validation
cv_scores = cross_val_score(adaboost_model, X, y, cv=5)  # cv=5 for 5-fold cross-validation

# Print the cross-validation scores
print("Cross-validation scores:", cv_scores)
print("Mean cross-validation score:", cv_scores.mean())

Adaboost Classifier

In [ ]:
# AdaBoost Classifier
adaboost_model = AdaBoostClassifier()
adaboost_model.fit(X_train, y_train_encoded)
adaboost_pred = adaboost_model.predict(X_test)
adaboost_accuracy = accuracy_score(y_test_encoded, adaboost_pred)
print("AdaBoost Accuracy:", adaboost_accuracy)
print("AdaBoost Classification Report:\n", classification_report(y_test_encoded, adaboost_pred))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Membuat daftar lengkap kelas
full_class_list = np.unique(np.concatenate((y_train_encoded, y_test_encoded)))

# Menghitung confusion matrix dengan daftar lengkap kelas
conf_matrix_adaboost = confusion_matrix(y_test_encoded, adaboost_pred, labels=full_class_list)

# Visualisasi confusion matrix sebagai gambar
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_adaboost, annot=True, fmt="d", cmap="Blues", 
            xticklabels=full_class_list,
            yticklabels=full_class_list)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix for AdaBoost Classifier')
plt.show()

Decision Tree

In [ ]:
# Decision Tree Classifier
decision_tree_model = DecisionTreeClassifier()
decision_tree_model.fit(X_train, y_train_encoded)
decision_tree_pred = decision_tree_model.predict(X_test)
decision_tree_accuracy = accuracy_score(y_test_encoded, decision_tree_pred)
print("Decision Tree Accuracy:", decision_tree_accuracy)
print("Decision Tree Classification Report:\n", classification_report(y_test_encoded, decision_tree_pred))

In [ ]:
from sklearn.tree import plot_tree

# Visualisasi decision tree
plt.figure(figsize=(20,10))
plot_tree(decision_tree_model, filled=True, feature_names=X_train.columns, class_names=np.unique(y_train_encoded).astype(str))
plt.show()

In [ ]:
# Menghitung confusion matrix dengan daftar lengkap kelas
conf_matrix_decision_tree = confusion_matrix(y_test_encoded, decision_tree_pred, labels=full_class_list)

# Visualisasi confusion matrix sebagai gambar
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_decision_tree, annot=True, fmt="d", cmap="Blues", 
            xticklabels=full_class_list,
            yticklabels=full_class_list)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix for Decision Tree Classifier')
plt.show()


Multinomial Naive Bayes

In [ ]:
# Multinomial Naive Bayes Classifier
naive_bayes_model = MultinomialNB()
naive_bayes_model.fit(X_train, y_train_encoded)
naive_bayes_pred = naive_bayes_model.predict(X_test)
naive_bayes_accuracy = accuracy_score(y_test_encoded, naive_bayes_pred)
print("Multinomial Naive Bayes Accuracy:", naive_bayes_accuracy)
print("Multinomial Naive Bayes Classification Report:\n", classification_report(y_test_encoded, naive_bayes_pred))

In [ ]:
# Menghitung confusion matrix dengan daftar lengkap kelas
conf_matrix_naive_bayes = confusion_matrix(y_test_encoded, naive_bayes_pred, labels=full_class_list)

# Visualisasi confusion matrix sebagai gambar
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_naive_bayes, annot=True, fmt="d", cmap="Blues", 
            xticklabels=full_class_list,
            yticklabels=full_class_list)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix for Multinomial Naive Bayes Classifier')
plt.show()


Prediksi

In [ ]:
# Contoh paragraf yang ingin diprediksi
paragraph = """Create Program Outcomes	"In order to establish Program Outcomes
As Curriculum Team,
I want the capability to add Program Outcomes"	"Scenario: Create Program Outcomes Successfully
Given I am on “http://127.0.0.1:8000/dashboard/curriculum”
When I press “Data”
And I press “Program Outcomes”
Then I should be on “Program Outcomes Page”
When I press “Add”
Then I should be on “Program Outcomes Creation Page”
When I fill in “Program Outcome Code” with “PO1”
And I fill in “Program Outcome Description” with “Graduates possess the ability to analyze, design, create, and comprehensively evaluate information systems in alignment with organizational goals, demonstrating effective proficiency.”
And I press “Add Program Outcome”
Then the response should contain “Success! Program Outcome has been added.”"	"Scenario: Failed to create Program Outcomes as the Program Outcome Code has already been taken.
Given I am on “http://127.0.0.1:8000/dashboard/curriculum”
When I press “Data”
And I press “Program Outcomes”
Then I should be on “Program Outcomes Page”
And the “Program Outcome Code” field should contain “PO1”
When I press “Add”
Then I should be on “Program Outcomes Creation Page”
When I fill in “Program Outcome Code” with “PO1”
And I fill in “Program Outcome Description” with “Graduates possess the ability to analyze, design, create, and comprehensively evaluate information systems in alignment with organizational goals, demonstrating effective proficiency.”
And I press “Add Program Outcome”
Then the response should contain “The Program Outcome Code has already been taken.”"	"Scenario: Failed to Create Program Outcomes Due to Empty Data Fields
Given I am on “http://127.0.0.1:8000/dashboard/curriculum”
When I press “Data”
And I press “Program Outcomes”
Then I should be on “Program Outcomes Page”
And the “Program Outcome Code” field should contain “PO1”
When I press “Add”
Then I should be on “Program Outcomes Creation Page”
When I fill in “Program Outcome Code” with “PO2”
And I press “Add Program Outcome”
Then the response should contain “Program Outcome Description field is required”."""
"

# Pra-pemrosesan teks (misalnya tokenisasi, menghapus stopwords, normalisasi)
# ...

# Vektorisasi paragraf
paragraph_vectorized = vectorizer.transform([paragraph])  # Menggunakan vektorisasi yang sama yang digunakan pada data latih

# Prediksi dengan model AdaBoost
predicted_label = adaboost_model.predict(paragraph_vectorized)

# Keluarkan hasil prediksi
print("Hasil Prediksi untuk Paragraf:")
print(predicted_label)


In [ ]:
# Splitting data into features (X) and target variable (y)
X = integrated_data.drop(columns=['Effort'])
y = integrated_data['Effort']

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Perform one-hot encoding on the target variable
onehot_encoder = OneHotEncoder()
y_train_encoded = onehot_encoder.fit_transform(y_train.values.reshape(-1,1)).toarray()  # Convert to dense array
y_test_encoded = onehot_encoder.transform(y_test.values.reshape(-1,1)).toarray()  # Convert to dense array

# Train AdaBoost model
adaboost_model = AdaBoostClassifier()
adaboost_model.fit(X_train, y_train_encoded)

# Predict
y_pred = adaboost_model.predict(X_test)

# Evaluate
accuracy = accuracy_score(y_test_encoded, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(y_test_encoded, y_pred))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# One-Hot Encoding for 'Effort'
onehot_encoder = OneHotEncoder()
effort_encoded = onehot_encoder.fit_transform(integrated_data['Effort'].values.reshape(-1,1))

# Splitting data into features (X) and target variable (y)
X = integrated_data.drop(columns=['Effort'])
y = effort_encoded

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model Creation and Evaluation

# AdaBoost Classifier
adaboost_model = AdaBoostClassifier()
adaboost_model.fit(X_train, y_train)
adaboost_pred = adaboost_model.predict(X_test)
adaboost_accuracy = accuracy_score(y_test, adaboost_pred)
print("AdaBoost Accuracy:", adaboost_accuracy)
print("AdaBoost Classification Report:\n", classification_report(y_test, adaboost_pred))

# Decision Tree Classifier
decision_tree_model = DecisionTreeClassifier()
decision_tree_model.fit(X_train, y_train)
decision_tree_pred = decision_tree_model.predict(X_test)
decision_tree_accuracy = accuracy_score(y_test, decision_tree_pred)
print("Decision Tree Accuracy:", decision_tree_accuracy)
print("Decision Tree Classification Report:\n", classification_report(y_test, decision_tree_pred))

# Multinomial Naive Bayes Classifier
naive_bayes_model = MultinomialNB()
naive_bayes_model.fit(X_train, y_train)
naive_bayes_pred = naive_bayes_model.predict(X_test)
naive_bayes_accuracy = accuracy_score(y_test, naive_bayes_pred)
print("Multinomial Naive Bayes Accuracy:", naive_bayes_accuracy)
print("Multinomial Naive Bayes Classification Report:\n", classification_report(y_test, naive_bayes_pred))


Case Folding

In [ ]:
df['Feature']=df['Feature'].str.lower()
df['User Story']=df['User Story'].str.lower()
df['Normal Flow']=df['Normal Flow'].str.lower()
df['Exception Flow']=df['Exception Flow'].str.lower()
df['Alternatif Flow']=df['Alternatif Flow'].str.lower()
label_mapping = {'Mudah': 0, 'Cukup': 1, 'Sulit': 2}
df['Label_Numeric'] = df['Score'].map(label_mapping)
df

Remove URL, html, dan punctuation

In [ ]:
import re 
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

def remove_URL(text):
    def replace_url(match):
        return ""

    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(replace_url, text)

# ternyata tidak ada sepertinya
def remove_html(text): 
    def replace_html(match):
        return ""
    html=re.compile(r'<.*?>')
    return html.sub(replace_html,text)

def remove_punctuation(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)

selected_columns = ['Feature', 'User Story', 'Normal Flow', 'Exception Flow', 'Alternatif Flow']

for col in selected_columns:
    for i in range(len(df[col])):
        df.at[i, col] = remove_URL(df.at[i, col])
        df.at[i, col] = remove_html(df.at[i, col])
        df.at[i, col] = remove_punctuation(df.at[i, col])

df

Spelling Correction (JANGAN DULU)

In [ ]:
from spellchecker import SpellChecker

spell = SpellChecker()

def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_word = spell.correction(word)
            if corrected_word is not None:
                corrected_text.append(corrected_word)
            else:
                # Handle the case where correction is None
                corrected_text.append(word)
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)

for col in selected_columns:
    for i in range(len(df[col])):
        df.at[i, col] = correct_spellings(df.at[i, col])


In [ ]:
text = "corect me plese"
correct_spellings(text)


Tokenizer

In [ ]:
import re 
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

selected_columns = ['Feature', 'User Story', 'Normal Flow', 'Exception Flow', 'Alternatif Flow']

for col in selected_columns:
    for i in range(len(df[col])):
        df.at[i, col] = nltk.word_tokenize(df.at[i, col])


df


Stopwords

In [ ]:
for col in selected_columns:
    for i in range(len(df[col])):
        first_text =  df.at[i, col]
        first_text_list_cleaned = [word for word in first_text if word.lower() not in stopwords]
        df.at[i, col] = first_text_list_cleaned
df
        

Stemming and Lemmatization

In [ ]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

lemm = WordNetLemmatizer()

for col in selected_columns:
    for i in range(len(df[col])):
        # Tokenize each text
        tokens = df.at[i, col]
        # Lemmatize each word
        lemmatized_tokens = [lemm.lemmatize(word) for word in tokens]
        # Join the lemmatized tokens back into a string
        lemmatized_text = ' '.join(lemmatized_tokens)
        # Update the DataFrame with the lemmatized text
        df.at[i, col] = lemmatized_text

df


Visualisasi

In [ ]:
import requests
import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image  # Tambahkan baris ini
import numpy as np
from wordcloud import WordCloud

url = "https://user-images.githubusercontent.com/74188336/142692890-641ebc21-2e47-4556-9d37-1c0b9e1a0587.jpeg"
response = requests.get(url)
img = Image.open(BytesIO(response.content))

text = ' '.join(df['Feature'].values)
mask = np.array(img)
wordcloud = WordCloud(max_font_size=50, max_words=1000, background_color="white", mask=mask, colormap='BuGn').generate(text.lower())
plt.figure(figsize=(5, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()


In [ ]:
url = "https://user-images.githubusercontent.com/74188336/142692894-c17240e4-1101-4591-9d10-71793e460816.jpeg"
response = requests.get(url)
img = Image.open(BytesIO(response.content))

text = ' '.join(df['User Story'].values)
mask = np.array(img)
wordcloud = WordCloud(max_font_size=50, max_words=1000, background_color="white", mask=mask, colormap='BuGn').generate(text.lower())
plt.figure(figsize=(5, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()


TF-IDF

In [ ]:
!pip install seaborn

Cross Validation

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

# Create and train the Multinomial Logistic Regression model
multinomial_model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
# Gunakan cross_val_predict untuk mendapatkan label prediksi untuk setiap lipatan cross-validation
y_pred_multinomial = cross_val_predict(multinomial_model, X_combined, y, cv=stratified_kfold)

# Hitung confusion matrix
conf_matrix_multinomial = confusion_matrix(y, y_pred_multinomial)

# Print confusion matrix
print("Confusion Matrix:")
print(conf_matrix_multinomial)

# Visualisasikan confusion matrix jika diperlukan
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_multinomial, annot=True, fmt="d", cmap="Blues", xticklabels=["Class 1", "Class 2", "Class 3"], yticklabels=["Class 1", "Class 2", "Class 3"])
plt.title('Confusion Matrix (Cross-Validation)')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

# Additional evaluation metrics
accuracy_multinomial = accuracy_score(y, y_pred_multinomial)
print(f"\nAccuracy for Multinomial Logistic Regression: {accuracy_multinomial:.2f}")

classification_report_multinomial = classification_report(y, y_pred_multinomial)
print("\nClassification Report for Multinomial Logistic Regression:")
print(classification_report_multinomial)


In [ ]:
# Random Forest
rf_model = RandomForestClassifier()

# Gunakan cross_val_predict dengan StratifiedKFold
y_pred_rf = cross_val_predict(rf_model, X_combined, y, cv=stratified_kfold)

# Hitung confusion matrix
conf_matrix_rf = confusion_matrix(y, y_pred_rf)

# Print confusion matrix
print("Confusion Matrix:")
print(conf_matrix_rf)

# Visualisasikan confusion matrix jika diperlukan
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_rf, annot=True, fmt="d", cmap="Blues", xticklabels=["Class 1", "Class 2", "Class 3"], yticklabels=["Class 1", "Class 2", "Class 3"])
plt.title('Confusion Matrix (Cross-Validation)')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

# Additional evaluation metrics
accuracy_rf = accuracy_score(y, y_pred_rf)
print(f"\nAccuracy for Random Forest: {accuracy_rf:.2f}")

classification_report_rf = classification_report(y, y_pred_rf)
print("\nClassification Report for Random Forest:")
print(classification_report_rf)

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import cross_val_predict, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier
from scipy.sparse import hstack
import matplotlib.pyplot as plt
import seaborn as sns

# Specify the number of folds (e.g., 5 or 10)
num_folds = 5

# Create a StratifiedKFold object with the desired number of splits
stratified_kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

# Menggabungkan semua fitur ke dalam matriks tunggal
df['Combined_Text'] = df['Feature'] + ' ' + df['User Story'] + ' ' + df['Normal Flow'] + ' ' + df['Exception Flow'] + ' ' + df['Alternatif Flow']
X_combined = TfidfVectorizer(max_features=5000).fit_transform(df['Combined_Text'])
X_combined = hstack([X_combined, df[['Jumlah Scenario', 'Jumlah kata']]])

# Memilih kolom-kolom yang akan digunakan sebagai label
y = df['Score']

# Inisialisasi model Linear SVM, Logistic Regression, dan Random Forest
svm_model = LinearSVC()
logistic_model = LogisticRegression()
rf_model = RandomForestClassifier()

# Membuat Stacking Ensemble dengan model-model yang telah diinisialisasi
stacking_model = StackingClassifier(
    estimators=[
        ('svm', svm_model),
        ('logistic', logistic_model),
        ('random_forest', rf_model)
    ],
    cv=stratified_kfold
)


# Gunakan cross_val_predict untuk mendapatkan label prediksi untuk setiap lipatan cross-validation
y_pred_stacking = cross_val_predict(stacking_model, X_combined, y.values.ravel(), cv=stratified_kfold)


# Hitung confusion matrix
conf_matrix_stacking = confusion_matrix(y, y_pred_stacking)

# Print confusion matrix
print("Confusion Matrix:")
print(conf_matrix_stacking)

# Visualisasikan confusion matrix jika diperlukan
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_stacking, annot=True, fmt="d", cmap="Blues", xticklabels=["Class 1", "Class 2", "Class 3"], yticklabels=["Class 1", "Class 2", "Class 3"])
plt.title('Confusion Matrix (Stacking Ensemble)')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

# Additional evaluation metrics
accuracy_stacking = accuracy_score(y, y_pred_stacking)
print(f"\nAccuracy for Stacking Ensemble: {accuracy_stacking:.2f}")

classification_report_stacking = classification_report(y, y_pred_stacking)
print("\nClassification Report for Stacking Ensemble:")
print(classification_report_stacking)


In [ ]:
pip install --upgrade scikit-learn

In [ ]:
from sklearn.ensemble import StackingClassifier
# Stacking classifier without final_estimator
stacking_model = StackingClassifier(
    estimators=[
        ('svm', svm_model),
        ('lr', logistic_model),
        ('rf', rf_model)
    ],
)

y_pred_stacking = cross_val_predict(stacking_model, X_combined, y.values.ravel(), cv=stratified_kfold)

# Hitung confusion matrix
conf_matrix_stacking = confusion_matrix(y, y_pred_stacking)

# Print confusion matrix
print("Confusion Matrix:")
print(conf_matrix_stacking)

# Visualisasikan confusion matrix jika diperlukan
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_stacking, annot=True, fmt="d", cmap="Blues", xticklabels=["Class 1", "Class 2", "Class 3"], yticklabels=["Class 1", "Class 2", "Class 3"])
plt.title('Confusion Matrix (Cross-Validation)')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

# Additional evaluation metrics
accuracy_stacking = accuracy_score(y, y_pred_stacking)
print(f"\nAccuracy for Ensemble Stacking Model: {accuracy_stacking:.2f}")

classification_report_stacking = classification_report(y, y_pred_stacking)
print("\nClassification Report for Ensemble Stacking Model:")
print(classification_report_stacking)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix
from scipy.sparse import hstack
import scipy
import matplotlib.pyplot as plt
import seaborn as sns

# Menggunakan MinMaxScaler
scaler = MinMaxScaler()

df['Combined_Text'] = df['Feature'] + ' ' + df['User Story'] + ' ' + df['Normal Flow'] + ' ' + df['Exception Flow'] + ' ' + df['Alternatif Flow']

# Membagi dataset menjadi data latih dan data uji
train, test = train_test_split(df, test_size=0.4)

# Memilih kolom-kolom yang akan digunakan sebagai fitur dan label
train_X = train[['Combined_Text']]
train_numeric = scaler.fit_transform(train[['Jumlah Scenario', 'Jumlah kata']])
train_y = train['Score']
test_X = test[['Combined_Text']]
test_y = test['Score']

# Feature extraction (TF-IDF) for each column on training set
tfidf_vectorizer_user_story = TfidfVectorizer(max_features=5000)
X_train = tfidf_vectorizer_user_story.fit_transform(train_X['Combined_Text'])

# Combine training set matrices
X_train_combined = hstack([X_train, train_numeric])

# Feature extraction (TF-IDF) for each column on test set
X_test = tfidf_vectorizer_user_story.transform(test_X['Combined_Text'])

# Combine test set matrices
X_test_combined = hstack([X_test, scaler.transform(test[['Jumlah Scenario', 'Jumlah kata']])])

# Model menggunakan LinearSVC
svm_model = LinearSVC()  
svm_model.fit(X_train_combined, train_y)
y_pred_svm = svm_model.predict(X_test_combined)

# Confusion Matrix for SVM
conf_matrix_svm = confusion_matrix(test_y, y_pred_svm)

# Plotting the heatmap for SVM
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_svm, annot=True, fmt="d", cmap="Blues", xticklabels=["Class 1", "Class 2", "Class 3"], yticklabels=["Class 1", "Class 2", "Class 3"])
plt.title('Confusion Matrix for Linear SVM')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

# Model menggunakan Adaboost dengan pengaturan algorithm='SAMME'
adaboost_model = AdaBoostClassifier(base_estimator=LinearSVC(), n_estimators=50, algorithm='SAMME', random_state=42)
adaboost_model.fit(X_train_combined, train_y)
y_pred_adaboost = adaboost_model.predict(X_test_combined)

# Confusion Matrix for Adaboost
conf_matrix_adaboost = confusion_matrix(test_y, y_pred_adaboost)

# Plotting the heatmap for Adaboost
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_adaboost, annot=True, fmt="d", cmap="Greens", xticklabels=["Class 1", "Class 2", "Class 3"], yticklabels=["Class 1", "Class 2", "Class 3"])
plt.title('Confusion Matrix for Adaboost with LinearSVC base learner (algorithm= SAMME)')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()



In [ ]:
from sklearn.model_selection import cross_val_score

# Define your models
svm_model = LinearSVC()
adaboost_model = AdaBoostClassifier(base_estimator=LinearSVC(), n_estimators=50, algorithm='SAMME', random_state=42)

# Combine features and numeric data
X_combined = hstack([tfidf_vectorizer_user_story.transform(df['Combined_Text']), scaler.transform(df[['Jumlah Scenario', 'Jumlah kata']])])

# Perform cross-validation for Linear SVM
svm_cv_scores = cross_val_score(svm_model, X_combined, df['Score'], cv=5)
print("Cross-Validation Scores for Linear SVM:", svm_cv_scores)
print("Mean Accuracy: {:.2f}%".format(svm_cv_scores.mean() * 100))

# Perform cross-validation for AdaBoost with LinearSVC base learner
adaboost_cv_scores = cross_val_score(adaboost_model, X_combined, df['Score'], cv=5)
print("\nCross-Validation Scores for AdaBoost with LinearSVC base learner:", adaboost_cv_scores)
print("Mean Accuracy: {:.2f}%".format(adaboost_cv_scores.mean() * 100))


In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# Inisialisasi model Multinomial Naive Bayes
nb_model = MultinomialNB()
# Melatih model pada data latih
nb_model.fit(X_train_combined, train_y)

# Membuat prediksi pada data uji
y_pred_nb = nb_model.predict(X_test_combined)

# Evaluasi model Naive Bayes pada data uji
accuracy_nb = accuracy_score(test_y, y_pred_nb)
conf_matrix_nb = confusion_matrix(test_y, y_pred_nb)

# Menampilkan hasil evaluasi
print("Accuracy of Naive Bayes: {:.2f}%".format(accuracy_nb * 100))
print("\nConfusion Matrix for Naive Bayes:\n", conf_matrix_nb)
print("\nClassification Report for Naive Bayes:\n", classification_report(test_y, y_pred_nb))

# Plotting Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_nb, annot=True, fmt="d", cmap="Blues", xticklabels=["Class 1", "Class 2", "Class 3"], yticklabels=["Class 1", "Class 2", "Class 3"])
plt.title('Confusion Matrix for Multinomial Naive Bayes')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

# Inisialisasi model AdaBoost dengan Naive Bayes sebagai base estimator
adaboost_model = AdaBoostClassifier(base_estimator=nb_model, n_estimators=50, random_state=42)

# Melatih model pada data latih
adaboost_model.fit(X_train_combined, train_y)

# Membuat prediksi pada data uji
y_pred_adaboost = adaboost_model.predict(X_test_combined)

# Evaluasi model AdaBoost dengan Naive Bayes pada data uji
accuracy_adaboost = accuracy_score(test_y, y_pred_adaboost)
conf_matrix_adaboost = confusion_matrix(test_y, y_pred_adaboost)

# Menampilkan hasil evaluasi
print("Accuracy of AdaBoost with Naive Bayes: {:.2f}%".format(accuracy_adaboost * 100))
print("\nConfusion Matrix for AdaBoost with Naive Bayes:\n", conf_matrix_adaboost)
print("\nClassification Report for AdaBoost with Naive Bayes:\n", classification_report(test_y, y_pred_adaboost))

# Plotting Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_adaboost, annot=True, fmt="d", cmap="Blues", xticklabels=["Class 1", "Class 2", "Class 3"], yticklabels=["Class 1", "Class 2", "Class 3"])
plt.title('Confusion Matrix for AdaBoost with Multinomial Naive Bayes')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Inisialisasi model Multinomial Naive Bayes
nb_model = MultinomialNB()

# Melatih model pada data latih
nb_model.fit(X_train_combined, train_y)

# Membuat prediksi pada data uji
y_pred_nb = nb_model.predict(X_test_combined)

# Evaluasi model Naive Bayes pada data uji
accuracy_nb = accuracy_score(test_y, y_pred_nb)
conf_matrix_nb = confusion_matrix(test_y, y_pred_nb)

# Menampilkan hasil evaluasi
print("Accuracy of Naive Bayes: {:.2f}%".format(accuracy_nb * 100))
print("\nConfusion Matrix for Naive Bayes:\n", conf_matrix_nb)
print("\nClassification Report for Naive Bayes:\n", classification_report(test_y, y_pred_nb))

# Plotting Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_nb, annot=True, fmt="d", cmap="Blues", xticklabels=["Class 1", "Class 2", "Class 3"], yticklabels=["Class 1", "Class 2", "Class 3"])
plt.title('Confusion Matrix for Multinomial Naive Bayes')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

# Inisialisasi model AdaBoost dengan Naive Bayes sebagai base estimator
adaboost_model = AdaBoostClassifier(base_estimator=nb_model, n_estimators=50, random_state=42)

# Perform cross-validation for AdaBoost with Naive Bayes
adaboost_cv_scores = cross_val_score(adaboost_model, X_train_combined, train_y, cv=5)
print("\nCross-Validation Scores for AdaBoost with Naive Bayes:", adaboost_cv_scores)
print("Mean Accuracy: {:.2f}%".format(adaboost_cv_scores.mean() * 100))

# Melatih model pada data latih (tanpa cross-validation)
adaboost_model.fit(X_train_combined, train_y)

# Membuat prediksi pada data uji
y_pred_adaboost = adaboost_model.predict(X_test_combined)

# Evaluasi model AdaBoost dengan Naive Bayes pada data uji
accuracy_adaboost = accuracy_score(test_y, y_pred_adaboost)
conf_matrix_adaboost = confusion_matrix(test_y, y_pred_adaboost)

# Menampilkan hasil evaluasi
print("\nAccuracy of AdaBoost with Naive Bayes: {:.2f}%".format(accuracy_adaboost * 100))
print("\nConfusion Matrix for AdaBoost with Naive Bayes:\n", conf_matrix_adaboost)
print("\nClassification Report for AdaBoost with Naive Bayes:\n", classification_report(test_y, y_pred_adaboost))

# Plotting Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_adaboost, annot=True, fmt="d", cmap="Blues", xticklabels=["Class 1", "Class 2", "Class 3"], yticklabels=["Class 1", "Class 2", "Class 3"])
plt.title('Confusion Matrix for AdaBoost with Multinomial Naive Bayes')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()


Coba tanpa konversi

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix
from scipy.sparse import hstack
import scipy
import matplotlib.pyplot as plt
import seaborn as sns

# Menggunakan MinMaxScaler
scaler = MinMaxScaler()

df['Combined_Text'] = df['Feature'] + ' ' + df['User Story'] + ' ' + df['Normal Flow'] + ' ' + df['Exception Flow'] + ' ' + df['Alternatif Flow']

# Membagi dataset menjadi data latih dan data uji
train, test = train_test_split(df, test_size=0.3)

# Memilih kolom-kolom yang akan digunakan sebagai fitur dan label
train_X = train[['Combined_Text']]
train_numeric = scaler.fit_transform(train[['Jumlah Scenario', 'Jumlah kata']])
train_y = train['Score']
test_X = test[['Combined_Text']]
test_y = test['Score']

# Feature extraction (TF-IDF) for each column on training set
tfidf_vectorizer_user_story = TfidfVectorizer(max_features=5000)
X_train = tfidf_vectorizer_user_story.fit_transform(train_X['Combined_Text'])

# Combine training set matrices
X_train_combined = hstack([X_train, train_numeric])

# Feature extraction (TF-IDF) for each column on test set
X_test = tfidf_vectorizer_user_story.transform(test_X['Combined_Text'])

# Combine test set matrices
X_test_combined = hstack([X_test, scaler.transform(test[['Jumlah Scenario', 'Jumlah kata']])])

# Model menggunakan LinearSVC
svm_model = LinearSVC()  
svm_model.fit(X_train_combined, train_y)

# Evaluate svm_model on the test set
y_pred_svm = svm_model.predict(X_test_combined)

# Confusion Matrix for SVM
conf_matrix_svm = confusion_matrix(test_y, y_pred_svm)

# Plotting the heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_svm, annot=True, fmt="d", cmap="Blues", xticklabels=["Class 1", "Class 2", "Class 3"], yticklabels=["Class 1", "Class 2", "Class 3"])
plt.title('Confusion Matrix for Linear SVM')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()


In [ ]:
from sklearn.naive_bayes import MultinomialNB

# Inisialisasi model Multinomial Naive Bayes
nb_model = MultinomialNB()

# Melatih model pada data latih
nb_model.fit(X_train_combined, train_y)

# Membuat prediksi pada data uji
y_pred_nb = nb_model.predict(X_test_combined)

# Evaluasi model Naive Bayes pada data uji
accuracy_nb = accuracy_score(test_y, y_pred_nb)
conf_matrix_nb = confusion_matrix(test_y, y_pred_nb)

# Menampilkan hasil evaluasi
print("Accuracy of Naive Bayes: {:.2f}%".format(accuracy_nb * 100))
print("\nConfusion Matrix for Naive Bayes:\n", conf_matrix_nb)
print("\nClassification Report for Naive Bayes:\n", classification_report(test_y, y_pred_nb))

# Plotting Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_nb, annot=True, fmt="d", cmap="Blues", xticklabels=["Class 1", "Class 2", "Class 3"], yticklabels=["Class 1", "Class 2", "Class 3"])
plt.title('Confusion Matrix for Multinomial Naive Bayes')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()


In [ ]:
a= pd.DataFrame(X_train.todense().T,
                index=tfidf_vectorizer_user_story.get_feature_names_out(),
                columns=[f'D{i+1}' for i in range (len(train_X))])
a

In [ ]:
X_train.dtype

In [ ]:
train_numeric.dtype

In [ ]:
train_numeric

In [ ]:
a= pd.DataFrame(train_numeric_sparse.todense().T,
                columns=[f'D{i+1}' for i in range (len(train_numeric))])
a

In [ ]:
a= pd.DataFrame(X_train_combined.todense().T,
                columns=[f'D{i+1}' for i in range (len(train_numeric))])
a

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from scipy.sparse import hstack
import scipy

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

df['Combined_Text'] = df['Feature'] + ' ' + df['User Story'] + ' ' + df['Normal Flow'] + ' ' + df['Exception Flow'] + ' ' + df['Alternatif Flow']

# Membagi dataset menjadi data latih dan data uji
train, test = train_test_split(df, test_size=0.3)

# Memilih kolom-kolom yang akan digunakan sebagai fitur dan label
train_X = train[['Combined_Text']]
train_numeric = scaler.fit_transform(train[['Jumlah Scenario']])
train_y = train['Score']
test_X = test[['Combined_Text']]
test_y = test['Score']

# Feature extraction (TF-IDF) for each column on training set
tfidf_vectorizer_user_story = TfidfVectorizer(max_features=5000)
X_train = tfidf_vectorizer_user_story.fit_transform(train_X['Combined_Text'])

# Convert matriks numerik train set menjadi matriks sparse
train_numeric_sparse = scipy.sparse.csr_matrix(train_numeric)

# Combine training set matrices
X_train_combined = hstack([X_train, train_numeric_sparse])

# Feature extraction (TF-IDF) for each column on test set
X_test = tfidf_vectorizer_user_story.transform(test_X['Combined_Text'])

# Convert matriks numerik test set menjadi matriks sparse
test_numeric_sparse = scipy.sparse.csr_matrix(scaler.transform(test[['Jumlah Scenario']]))

# Combine test set matrices
X_test_combined = hstack([X_test, test_numeric_sparse])


svm_model = LinearSVC()  
svm_model.fit(X_train_combined, train_y)

# Evaluate svm_model on the test set
y_pred_svm = svm_model.predict(X_test_combined)

# Confusion Matrix for SVM
conf_matrix_svm = confusion_matrix(test_y, y_pred_svm)


import matplotlib.pyplot as plt
import seaborn as sns

# Plotting the heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_svm, annot=True, fmt="d", cmap="Blues", xticklabels=["Class 1", "Class 2", "Class 3"], yticklabels=["Class 1", "Class 2", "Class 3"])
plt.title('Confusion Matrix for Linear SVM')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()



In [ ]:
conf_matrix_svm

In [ ]:
# Create and train the Logistic Regression model
logistic_model = LogisticRegression()
logistic_model.fit(X_train_combined, train_y)

# Evaluate Logistic Regression on the test set
y_pred_logistic = logistic_model.predict(X_test_combined)

# Confusion Matrix for Logistic Regression
conf_matrix_logistic = confusion_matrix(test_y, y_pred_logistic)
print("Confusion Matrix for Logistic Regression:")
print(conf_matrix_logistic)

# Additional evaluation metrics
accuracy_logistic = accuracy_score(test_y, y_pred_logistic)
print(f"\nAccuracy for Logistic Regression: {accuracy_logistic:.2f}")

classification_report_logistic = classification_report(test_y, y_pred_logistic)
print("\nClassification Report for Logistic Regression:")
print(classification_report_logistic)

In [ ]:
# Random Forest
rf_model = RandomForestClassifier()
rf_model.fit(X_train_combined, train_y)

# Evaluate Random Forest on the test set
y_pred_rf = rf_model.predict(X_test_combined)

# Confusion Matrix for Random Forest
conf_matrix_rf = confusion_matrix(test_y, y_pred_rf)
print("\nConfusion Matrix for Random Forest:")
print(conf_matrix_rf)


In [ ]:
# Stacking classifier without final_estimator
stacking_model = StackingClassifier(
    estimators=[
        ('svm', svm_model),
        ('lr', logistic_model),
        ('rf', rf_model)
    ],
    cv=2  # Adjust the number of splits as needed
)

# Train the stacking model
stacking_model.fit(X_train_combined, train_y)

# Evaluate stacking model on the test set
y_pred_stacking = stacking_model.predict(X_test_combined)

# Confusion Matrix for Stacking
conf_matrix_stacking = confusion_matrix(test_y, y_pred_stacking)
print("Confusion Matrix for Stacking:")
print(conf_matrix_stacking)

# Additional evaluation metrics
accuracy_stacking = accuracy_score(test_y, y_pred_stacking)
print(f"\nAccuracy for Stacking: {accuracy_stacking:.2f}")

classification_report_stacking = classification_report(test_y, y_pred_stacking)
print("\nClassification Report for Stacking:")
print(classification_report_stacking)

Menampilkan matriks sparse

In [ ]:
a= pd.DataFrame(X_train.todense().T,
                index=tfidf_vectorizer_user_story.get_feature_names_out(),
                columns=[f'D{i+1}' for i in range (len(train_X))])
a

In [ ]:
a= pd.DataFrame(train_numeric_sparse.todense().T,
                columns=[f'D{i+1}' for i in range (len(train_numeric))])
a

In [ ]:
a= pd.DataFrame(X_train_combined.todense().T,
                columns=[f'D{i+1}' for i in range (len(train_numeric))])
a